In [14]:
import pandas as pd

In [15]:
df = pd.read_csv('cleaned_DB_3.csv')
df.head()

,user_id,age,gender,occupation,work_mode,screen_time_hours,work_screen_hours,leisure_screen_hours,sleep_hours,sleep_quality_1_5,stress_level_0_10,productivity_0_100,exercise_minutes_per_week,social_hours_per_week,mental_wellness_index_0_100
0,U0001,33,Female,Employed,Remote,10.79,5.44,5.35,6.63,1.0,9.3,44.7,127,0.7,9.3
1,U0002,28,Female,Employed,In-person,7.40,0.37,7.03,8.05,3.0,5.7,78.0,74,2.1,56.2
2,U0003,35,Female,Employed,Hybrid,9.78,1.09,8.69,6.48,1.0,9.1,51.8,67,8.0,3.6
3,U0004,42,Male,Employed,Hybrid,11.13,0.56,10.57,6.89,1.0,10.0,37.0,0,5.7,0.0
4,U0005,28,Male,Student,Remote,13.22,4.09,9.13,5.79,1.0,10.0,38.5,143,10.1,0.0


In [16]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

In [17]:
def classification_training(data):
    """
    Функция для обучения моделей классификации на данных о ментальном здоровье
    
    Args:
        data (pd.DataFrame): Обработанный датасет из второй лабораторной работы
    """
    
    # 1. Создаем копию данных чтобы не изменять оригинальный датафрейм
    df = data.copy()
    
    # 2. Трансформируем целевую переменную в бинарную
    # значения < 15 -> 0, >= 15 -> 1
    df['mental_wellness_binary'] = np.where(df['mental_wellness_index_0_100'] < 15, 0, 1)
    
    # 3. Подготавливаем признаки и целевую переменную
    # Исключаем нечисловые колонки и саму целевую переменную
    exclude_cols = ['user_id', 'gender', 'occupation', 'work_mode', 
                   'mental_wellness_index_0_100', 'mental_wellness_binary']
    
    feature_cols = [col for col in df.columns if col not in exclude_cols]
    
    X = df[feature_cols]
    y = df['mental_wellness_binary']
    
    # 4. Разделяем данные на обучающую и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # 5. Масштабируем признаки
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # 6. Обучаем модели
    
    # Модель SVM с подобранными гиперпараметрами
    svm_model = SVC(
        C=1.0,           # Параметр регуляризации
        kernel='rbf',     # Радиальная базисная функция
        gamma='scale',    # Автоматический подбор gamma
        random_state=42
    )
    svm_model.fit(X_train_scaled, y_train)
    
    # Модель логистической регрессии с подобранными гиперпараметрами
    lr_model = LogisticRegression(
        C=1.0,           # Параметр регуляризации
        penalty='l2',     # L2 регуляризация
        solver='liblinear', # Алгоритм оптимизации
        random_state=42,
        max_iter=1000     # Увеличиваем количество итераций
    )
    lr_model.fit(X_train_scaled, y_train)
    
    # 7. Предсказания и оценка моделей
    
    # SVM предсказания
    svm_pred = svm_model.predict(X_test_scaled)
    svm_accuracy = accuracy_score(y_test, svm_pred)
    svm_f1 = f1_score(y_test, svm_pred)
    
    # Логистическая регрессия предсказания
    lr_pred = lr_model.predict(X_test_scaled)
    lr_accuracy = accuracy_score(y_test, lr_pred)
    lr_f1 = f1_score(y_test, lr_pred)
    
    # 8. Вывод результатов 
    print(f"SVM: {svm_accuracy:.4f}; {svm_f1:.4f}")
    print(f"LR: {lr_accuracy:.4f}; {lr_f1:.4f}")

In [18]:
classification_training(df)

SVM: 0.8500; 0.8500
LR: 0.8875; 0.8916
